#### The Objective of this Project is to scrape a Corpus of songs for few singers and store them locally. Later the songs are read and a Prediction model has been built on it. 

# Building the Raw Corpus

In [1]:
import requests
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import re
disable_warnings(InsecureRequestWarning)

In [ ]:
from bs4 import BeautifulSoup
import os
corpus = []
labels = []
i = 0
for fn in os.listdir('songs/'):
    with open('songs/' + fn, 'r', encoding = 'utf-8') as file:
        file_soup = BeautifulSoup(file, "html.parser")
        result = file_soup.find('pre', class_="lyric-body")
        
        
        if result:
            title = file_soup.find('title')
            label = str(title.text)
            print(title.text)
            labels.append(label[:5])
            corpus.append(result.text)
            i = i + 1
        else:
            continue
print(f' {i} files added to the Corpus')

Madonna - Music Lyrics | Lyrics.com
Madonna - Frozen Lyrics | Lyrics.com
Madonna - Back That Up to the Beat - demo [Demo] Lyrics | Lyrics.com
Madonna - Back That Up to the Beat - sped up [Sped Up] Lyrics | Lyrics.com
Bryan Adams - One Night Love Affair Lyrics | Lyrics.com
Bryan Adams - She's Only Happy When She's Dancing Lyrics | Lyrics.com
Bryan Adams - Run To You Lyrics | Lyrics.com
Bryan Adams - Heaven Lyrics | Lyrics.com
Bryan Adams - Somebody Lyrics | Lyrics.com
Bryan Adams - Kids Wanna Rock Lyrics | Lyrics.com
Bryan Adams - It's Only Love Lyrics | Lyrics.com
Bryan Adams - Long Gone Lyrics | Lyrics.com
Bryan Adams - Ain't Gonna Cry Lyrics | Lyrics.com
Jive Bunny & the Mastermixers - Music Lyrics | Lyrics.com
Jive Bunny & the Mastermixers - Music [Radyo Versiyonu] Lyrics | Lyrics.com
Madonna - Lucky Star Lyrics | Lyrics.com
Madonna - Borderline Lyrics | Lyrics.com
Madonna - Burning Up Lyrics | Lyrics.com
Madonna - I Know It Lyrics | Lyrics.com
Madonna - Holiday Lyrics | Lyrics.com


In [ ]:
print(title.text)

In [4]:
print(len(corpus))

2966


In [ ]:
print(corpus[100])

In [ ]:
print(labels[100])

In [ ]:
print(len(labels))

In [ ]:
labels_df = pd.DataFrame(data = labels, columns = ['artist'])
labels_df.value_counts()

# Vectorization and Normalizing the Corpus

In [ ]:
import string # useful for among others getting all punctuations!

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordDetokenizer

from sklearn.feature_extraction import _stop_words as stop_words
skl_stopwords = stop_words.ENGLISH_STOP_WORDS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
lemmatizer = WordNetLemmatizer()
tokenizer = TreebankWordDetokenizer()

In [ ]:
def tokenize_lemmatize(text, stopwords=skl_stopwords, tokenizer=tokenizer, lemmatizer=lemmatizer):
    text = "".join([i for i in text if i not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords]

In [ ]:
corpus[2788]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer(tokenizer=tokenize_lemmatize, stop_words=None)
X = vectorizer.fit_transform(corpus)
X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names_out(), index=labels)


#df_idf.sort_values(by=['idf_weights'])

In [ ]:
tf = TfidfTransformer()
# Use the CountVectorized data
x_norm = tf.fit_transform(X_df)
X_norm_df=pd.DataFrame(x_norm.todense(), columns=vectorizer.get_feature_names_out(), index=labels)
X_norm_df
X_norm_df.max()
#X_norm_df.columns[100]

In [ ]:
type(X_norm_df)

In [ ]:
Xtest

# Train the Model

In [ ]:
#Split the data into Training and testing datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


Xtrain, Xtest, ytrain, ytest = train_test_split(X_norm_df, labels, random_state=42)
Xtrain.shape, Xtest.shape

#### Logistic Regression

In [ ]:
Xtrain.head(5)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
m_lgr = LogisticRegression()
m_lgr.fit(Xtrain, ytrain)


m_lgr_trainscore = m_lgr.score(Xtrain,ytrain)
m_lgr_testscore = m_lgr.score(Xtest,ytest)


ypred_lgr = m_lgr.predict(X_norm_df)
m_lgr_precision = precision_score(labels, ypred, average='micro')
m_lgr_recall = recall_score(labels, ypred, average='micro')
m_lgr_fscore = f1_score(labels, ypred_lgr, average='micro')

print(f'Training Score {m_lgr_trainscore}')
print(f'Testing Score {m_lgr_testscore}')
print(f'Precision Score {m_lgr_precision}')
print(f'Recall Score {m_lgr_recall}')
print(f'F1 Score {m_lgr_fscore}')

In [ ]:
import seaborn as sns
def plot_heatmap(cm):
    
    """
    Function that plots a heatmap based on a correlation matrix
    """
    
    fig = plt.figure(figsize=(6, 6))
    ax = plt.subplot()
    sns.heatmap(cm, annot=True,  cbar=True, \
                cmap = 'BuPu', vmax = 200,\
               xticklabels=['Madonna', 'Bryan Adams'],\
                yticklabels=['Madonna', 'Bryan Adams'],\
                linecolor='white',\
               linewidth=2, square = True, fmt = '.0f', robust = True); 
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    ax.set_xticklabels(['Madonna','Bryan Adams'])

In [ ]:
len(ytrain)

#### Naive Bayes Classifier

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # import multinomial Naive Bayes

In [ ]:
# Train the NB Model
import nltk.metrics
nb = MultinomialNB()
nb.fit(Xtrain,  ytrain)

nb_trainscore = nb.score(Xtrain,ytrain)
nb_testscore = nb.score(Xtest,ytest)

ypred = nb.predict(X_norm_df)
nb_precision = precision_score(labels, ypred, average='micro')
nb_recall = recall_score(labels, ypred, average='micro')
nb_fscore = f1_score(labels, ypred, average='micro')

print(f'Training Score {nb_trainscore}')
print(f'Testing Score {nb_testscore}')
print(f'Precision Score {nb_precision}')
print(f'Recall Score {nb_recall}')
print(f'F1 Score {nb_fscore}')

#### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 20, max_depth = 3, random_state =10)
rf.fit(Xtrain, ytrain)


rf_trainscore = rf.score(Xtrain,ytrain)
rf_testscore = rf.score(Xtest,ytest)

ypred = rf.predict(X_norm_df)
rf_precision = precision_score(labels, ypred, average='micro')
rf_recall = recall_score(labels, ypred, average='micro')
rf_fscore = f1_score(labels, ypred, average='micro')

print(f'Training Score {rf_trainscore}')
print(f'Testing Score {rf_testscore}')
print(f'Precision Score {rf_precision}')
print(f'Recall Score {rf_recall}')
print(f'F1 Score {rf_fscore}')

## Handle Imbalance Classes

In [ ]:
labels_df = pd.DataFrame(data = labels, columns = ['artist'])
labels_df.value_counts()

In [ ]:
labels_df.value_counts()

# Summary of the Results

In [ ]:
pd.DataFrame({'Train Score':[m_lgr_trainscore, nb_trainscore,rf_trainscore],
              'Test Score':[m_lgr_testscore, nb_testscore,rf_testscore],
            'Precision':[m_lgr_precision, nb_precision, rf_precision],
              "recall":[m_lgr_recall, nb_recall, rf_recall],
             "F1Score":[m_lgr_fscore, nb_fscore, rf_fscore]},
              index=['Logistic Regression','Naive Bayes','Random Forest'])

# Word Cloud

In [ ]:
from wordcloud import ImageColorGenerator
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = " ".join(i for i in X_df.columns)
stopwords = set(skl_stopwords)
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()